<a href="https://colab.research.google.com/github/shrirang20/figr.design-task/blob/master/Task-2-Finetunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch transformers nltk

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW, AutoConfig, AutoModelForSeq2SeqLM
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import traceback
import logging
from nltk.translate.bleu_score import sentence_bleu

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
df = pd.read_csv('/content/websight_data.csv')

In [4]:
df.head()

,image,text,llm_generated_idea
0,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"<html>\n<link href=""https://cdn.jsdelivr.net/n...",Fashion Brand: A visually stunning layout with...
1,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"<html>\n<link href=""https://cdn.jsdelivr.net/n...",Restaurant Chain: A design with a mouth-wateri...
2,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"<html>\n<link href=""https://cdn.jsdelivr.net/n...","Consulting Firm: A clean, professional design ..."
3,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"<html>\n<link href=""https://cdn.jsdelivr.net/n...",Real Estate Agency: A user-friendly design wit...
4,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"<html>\n<link href=""https://cdn.jsdelivr.net/n...","Education Platform: A design with a wide, hero..."


In [ ]:
df['text'][0]

'<html>\n<link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">\n<body class="bg-gray-100">\n  <div class="flex flex-col items-center justify-center h-screen">\n    <h1 class="text-4xl font-bold text-center text-gray-800">Fashion Brand</h1>\n    <p class="mt-4 text-lg text-center text-gray-600">\n      Fashion Brand is a leading fashion brand that offers a wide range of high-quality, stylish clothing and accessories. Our collections are designed to reflect the latest fashion trends and are made from high-quality materials. We are committed to providing our customers with the best possible shopping experience and we strive to make our collections accessible to all.\n    </p>\n    <div class="mt-8">\n      <img src="https://source.unsplash.com/random/300x200/?fashion" alt="Fashion Brand" class="rounded">\n    </div>\n    <div class="mt-8">\n      <nav class="flex justify-center space-x-4">\n        <a href="#" class="px-4 py-2 text-white bg-g

In [5]:
# Create a custom dataset class
class CodeDataset(Dataset):
    def __init__(self, prompts, codes, tokenizer, max_length=512):
        self.prompts = prompts
        self.codes = codes
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.prompts)

    def __getitem__(self, idx):
        prompt = self.prompts[idx]
        code = self.codes[idx]
        encoding = self.tokenizer(prompt, truncation=True,
                                  padding='max_length',
                                  max_length=self.max_length,
                                  return_tensors='pt')
        target_encoding = self.tokenizer(code, truncation=True,
                                         padding='max_length',
                                         max_length=self.max_length,
                                         return_tensors='pt')
        return {'input_ids': encoding['input_ids'].squeeze(),
                'attention_mask': encoding['attention_mask'].squeeze(),
                'labels': target_encoding['input_ids'].squeeze()}


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:",device)

device: cuda


In [7]:
# Initialize tokenizer and model
model_name = 'Salesforce/codet5-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/codet5-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [8]:
model.save_pretrained('/content/pre-trained-model')

In [13]:
# Prepare dataset and dataloader
prompts = df['llm_generated_idea'].tolist()
codes = df['text'].tolist()
train_prompts,val_prompts,train_codes,val_codes=train_test_split(prompts,
                                                                 codes,
                                                                 test_size=0.1)
train_dataset=CodeDataset(train_prompts, train_codes, tokenizer,max_length=256)
val_dataset = CodeDataset(val_prompts, val_codes, tokenizer, max_length=256)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=4)

In [ ]:
prompts[0]

'Fashion Brand: A visually stunning layout with a full-width, rotating image carousel showcasing their latest collections, a bold, center-aligned logo, and a bottom navigation menu. The color palette is inspired by the latest fashion trends.'

In [9]:
# Setup optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [10]:
# Training loop
from torch.cuda.amp import GradScaler, autocast

scaler = GradScaler()

def train(model, dataloader, optimizer, scaler, accumulation_steps=1):
    model.train()
    optimizer.zero_grad()

    for i,batch in enumerate(dataloader):
        with autocast():
          input_ids = batch['input_ids'].to(model.device)
          attention_mask = batch['attention_mask'].to(model.device)
          labels = batch['labels'].to(model.device)

          outputs=model(input_ids,attention_mask=attention_mask,labels=labels)
          loss = outputs.loss
          loss = loss / accumulation_steps
          scaler.scale(loss).backward()

        if (i+1)% accumulation_steps ==0:
          scaler.step(optimizer)
          scaler.update()
          optimizer.zero_grad()

In [11]:
# Validation loop
def evaluate(model, dataloader):
    model.eval()
    total_loss = 0
    all_references = []
    all_hypotheses = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(model.device)
            attention_mask = batch['attention_mask'].to(model.device)
            labels = batch['labels'].to(model.device)

            outputs=model(input_ids,attention_mask=attention_mask,labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            # Decode the generated code
            generated_codes=tokenizer.batch_decode(outputs.logits.argmax(dim=-1),
                                                  skip_special_tokens=True)
            references=tokenizer.batch_decode(labels, skip_special_tokens=True)

            all_references.extend(references)
            all_hypotheses.extend(generated_codes)

    bleu_score=sum(sentence_bleu([ref.split()], hyp.split()) for ref,
              hyp in zip(all_references, all_hypotheses)) / len(all_references)

    return total_loss / len(dataloader), bleu_score

In [14]:
# Execute training and evaluation
for epoch in range(3):  # Number of epochs
    train(model, train_dataloader, optimizer, scaler, accumulation_steps=1)
    val_loss = evaluate(model, val_dataloader)
    print(f'Epoch {epoch + 1}: Validation Loss = {val_loss}')


Epoch 1: Validation Loss = (1.0391722297668458, 0.1486174651125726)
Epoch 2: Validation Loss = (0.7855813157558441, 0.25278321284248795)
Epoch 3: Validation Loss = (0.6859173536300659, 0.2982866427097455)


In [15]:
model.save_pretrained('/content/Fine-tuned-model-weights')

In [16]:
save_directory = 'fine-tuned-model'
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

# Save the fine-tuned model
print("Saving Fine-tuned-model")

model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)
print("model Saved at", save_directory)
logging.info("Model saved at %s", save_directory)

Saving Fine-tuned-model
model Saved at fine-tuned-model


In [ ]:
model.save_pretrained(save_directory)